In [ ]:
!nvidia-smi

## 构建目录

In [ ]:
!rm -rf /home/mw/project/*
!cp -r /home/mw/input/gcode_2022052329197/项目代码-202205232/gaiic2022-track2-commodity-title-entity-recognition/* /home/mw/project/
!cp -r /home/mw/input/track2_contest_5713/train_data/train_data/ /home/mw/project/data/contest_data/
!mkdir /home/mw/project/data/contest_data/preliminary_test_a/
!mkdir /home/mw/project/data/contest_data/preliminary_test_b/
!cp -r /home/mw/input/track2_contest_5713/preliminary/*_A.txt /home/mw/project/data/contest_data/preliminary_test_a/
!cp -r /home/mw/input/track2_contest_5713/preliminary/*_B.txt /home/mw/project/data/contest_data/preliminary_test_b/

## 安装apex

In [ ]:
%cd /home/mw/project/apex
!pip install -v --disable-pip-version-check --no-cache-dir ./ --user
%cd /home/mw/project/
!rm -rf apex/

## 继续预训练（阶段一）

In [ ]:
%cd /home/mw/project/code
!rm -rf /home/mw/project/data/pretrain_model/nezha_pretrain_v2/
!python run_pretrain_nezha_v2.py

In [ ]:
!ls /home/mw/temp/nezha_pretrain_v2/checkpoint-36000/

In [ ]:
!mkdir /home/mw/project/data/pretrain_model/nezha_pretrain_v2/
!cp -r /home/mw/temp/nezha_pretrain_v2/checkpoint-36000/ /home/mw/project/data/pretrain_model/nezha_pretrain_v2/

## 继续预训练（阶段二）

In [ ]:
%cd /home/mw/project/code
!rm -rf /home/mw/project/data/pretrain_model/nezha_pretrain_v3/
!python run_pretrain_nezha_v3.py

In [ ]:
!ls /home/mw/temp/nezha_pretrain_v3/checkpoint-7800/

In [ ]:
!mkdir /home/mw/project/data/pretrain_model/nezha_pretrain_v3/
!cp -r /home/mw/temp/nezha_pretrain_v3/checkpoint-7800/ /home/mw/project/data/pretrain_model/nezha_pretrain_v3/

## 模型微调

In [ ]:
!python kfold_split_data.py
!ls /home/mw/temp/10_folds_data/

In [ ]:
%cd /home/mw/project/code
!python exp_gaiic_global_pointer_v2.py \
   --experiment_code=experiment_bert_base_fold0_gp_v2_pre_v73 \
   --task_name=gaiic \
   --model_type=nezha \
   --do_lower_case \
   --pretrained_model_path=/home/mw/temp/nezha_pretrain_v3/checkpoint-7800/ \
   --data_dir=/home/mw/temp/10_folds_data/ \
   --train_input_file=train.all.jsonl \
   --eval_input_file=dev.0.jsonl \
   --output_dir=/home/mw/project/data/model_data/ \
   --do_train \
   --save_best \
   --test_input_file=../test_submit_dev_0.txt \
   --eval_checkpoint_path=/home/mw/project/data/best_model \
   --submit_file_path=/home/mw/project/results.txt \
   --evaluate_during_training \
   --train_max_seq_length=128 \
   --eval_max_seq_length=128 \
   --test_max_seq_length=128 \
   --per_gpu_train_batch_size=16 \
   --per_gpu_eval_batch_size=32 \
   --per_gpu_test_batch_size=32 \
   --gradient_accumulation_steps=1 \
   --learning_rate=3e-5 \
   --other_learning_rate=1e-3 \
   --weight_decay=0.001 \
   --scheduler_type=cosine \
   --base_model_name=bert \
   --warmup_proportion=0.1 \
   --max_grad_norm=1.0 \
   --num_train_epochs=10 \
   --use_rope \
   --do_lstm \
   --do_fgm \
   --num_lstm_layers=2 \
   --adam_epsilon=1e-8 \
   --post_lstm_dropout=0.5 \
   --inner_dim=64 \
   --loss_type=pcl \
   --pcl_epsilon=2.5 \
   --pcl_alpha=1.5 \
   --do_awp \
   --awp_epoch=2 \
   --awp_lr=0.1 \
   --do_rdrop \
   --rdrop_weight=0.4 \
   --rdrop_epoch=1 \
   --seed=42 \
   --fp16